**Not A Demo Notebook**
# Benchmarking workflows

- Univariate with Exogenous forecasting
- Hierarchical forecasting
- Multivariate forecasting

### Robust Model Performance Evaluation

In this notebook, we will demonstrate how to evaluate the performance of a model using time series cross-validation.

1. Data preparation
2. Model evaluation for different time series tasks
    - using splitter to split data into multiple windows
    - evaluate prediction performance for each window and across windows
3. M4 Simple Workflows

In [1]:
import pandas as pd

from sktime.forecasting.model_evaluation import evaluate
from sktime.forecasting.model_selection import (
    ExpandingGreedySplitter,
    ExpandingWindowSplitter,
    SingleWindowSplitter,
    SlidingWindowSplitter,
)
from sktime.forecasting.naive import NaiveForecaster
from sktime.performance_metrics.forecasting import (
    MeanAbsolutePercentageError,
    MeanAbsoluteScaledError,
    MeanSquaredError,
)
from sktime.utils._testing.hierarchical import _make_hierarchical
from sktime.utils._testing.series import _make_series
from sktime.utils._testing.panel import _make_panel

In [7]:
# 3 unique series per level of hierarchy (2 levels)
y_hierarchical = _make_hierarchical(hierarchy_levels=(2, 3), random_state=0)
# 4 multivariate series
y_multivariate = _make_series(n_timepoints=12, n_columns=4, random_state=0)
y = _make_series(n_timepoints=12, random_state=0)
y_panel = _make_panel(n_timepoints=12, n_instances=6, random_state=0)

In [8]:
fh = [1, 2]
step_length = 2
window_length = 8
fh_timedelta = pd.timedelta_range(
    start="1 day", end="2 days", freq="D"
)  # pd.Timedelta(2, unit='D')

# each unique series (lowest hierachical level) should have two splits
cv = ExpandingWindowSplitter(
    initial_window=window_length, fh=fh, step_length=step_length
)
sliding = SlidingWindowSplitter(
    fh=fh, window_length=window_length, step_length=step_length, start_with_window=True
)
single = SingleWindowSplitter(fh=fh_timedelta, window_length=pd.offsets.Day(8))

In [9]:
for train, test in single.split(y_hierarchical):
    print(train)
    print(test)
    print("------------------")
    print(y_hierarchical.iloc[test])
    print()

[ 2  3  4  5  6  7  8  9 14 15 16 17 18 19 20 21 26 27 28 29 30 31 32 33
 38 39 40 41 42 43 44 45 50 51 52 53 54 55 56 57 62 63 64 65 66 67 68 69]
[10 11 22 23 34 35 46 47 58 59 70 71]
------------------
                            c0
h0   h1   time                
h0_0 h1_0 2000-01-11  3.697033
          2000-01-12  5.007263
     h1_1 2000-01-11  4.417426
          2000-01-12  2.810825
     h1_2 2000-01-11  3.205078
          2000-01-12  3.709339
h0_1 h1_0 2000-01-11  2.300194
          2000-01-12  4.330480
     h1_1 2000-01-11  2.918668
          2000-01-12  3.190249
     h1_2 2000-01-11  4.282080
          2000-01-12  3.681973



In [17]:
list(cv.split(y_hierarchical))

[(array([ 0,  1,  2,  3,  4,  5,  6,  7, 12, 13, 14, 15, 16, 17, 18, 19, 24,
         25, 26, 27, 28, 29, 30, 31, 36, 37, 38, 39, 40, 41, 42, 43, 48, 49,
         50, 51, 52, 53, 54, 55, 60, 61, 62, 63, 64, 65, 66, 67],
        dtype=int64),
  array([ 8,  9, 20, 21, 32, 33, 44, 45, 56, 57, 68, 69])),
 (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 12, 13, 14, 15, 16, 17, 18,
         19, 20, 21, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 36, 37, 38, 39,
         40, 41, 42, 43, 44, 45, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 60,
         61, 62, 63, 64, 65, 66, 67, 68, 69], dtype=int64),
  array([10, 11, 22, 23, 34, 35, 46, 47, 58, 59, 70, 71]))]

In [18]:
list(sliding.split(y_hierarchical))

[(array([ 0,  1,  2,  3,  4,  5,  6,  7, 12, 13, 14, 15, 16, 17, 18, 19, 24,
         25, 26, 27, 28, 29, 30, 31, 36, 37, 38, 39, 40, 41, 42, 43, 48, 49,
         50, 51, 52, 53, 54, 55, 60, 61, 62, 63, 64, 65, 66, 67],
        dtype=int64),
  array([ 8,  9, 20, 21, 32, 33, 44, 45, 56, 57, 68, 69])),
 (array([ 2,  3,  4,  5,  6,  7,  8,  9, 14, 15, 16, 17, 18, 19, 20, 21, 26,
         27, 28, 29, 30, 31, 32, 33, 38, 39, 40, 41, 42, 43, 44, 45, 50, 51,
         52, 53, 54, 55, 56, 57, 62, 63, 64, 65, 66, 67, 68, 69],
        dtype=int64),
  array([10, 11, 22, 23, 34, 35, 46, 47, 58, 59, 70, 71]))]

In [10]:
hierachical_scorer = MeanSquaredError(multilevel="uniform_average_time")
forecaster = NaiveForecaster(strategy="last")


def manual_evaluate(cv, y, fh, forecaster, scores):
    errors = []
    for train, test in cv.split_series(y):
        forecaster.fit(train)
        y_pred = forecaster.predict(fh)
        error = scores(test, y_pred)
        errors.append(error)

    for i, error in enumerate(errors):
        print(f"window/fold {i}: {error}")
    return errors


cv_errors = manual_evaluate(cv, y_hierarchical, fh, forecaster, hierachical_scorer)

window/fold 0: 2.216486065005522
window/fold 1: 1.1927155356395638


In [9]:
# or you can just use evaluate function!
backtest = evaluate(
    forecaster=forecaster,
    y=y_hierarchical,
    cv=cv,
    scoring=hierachical_scorer,
    return_data=True,
    error_score="raise",
)
backtest

,test_MeanSquaredError,fit_time,pred_time,len_train_window,cutoff,y_train,y_test,y_pred
0,2.216486,0.077827,0.144058,48,2000-01-08 00:00:00,c0 h0 h1 time ...,c0 h0 h1 time ...,c0 h0 h1 time ...
1,1.192716,0.052101,0.136132,60,2000-01-10 00:00:00,c0 h0 h1 time ...,c0 h0 h1 time ...,c0 h0 h1 time ...


In [11]:
# checking train data
backtest["y_train"][0]

c0
h0   h1   time                
h0_0 h1_0 2000-01-01  5.317042
          2000-01-02  3.953147
          2000-01-03  4.531728
          2000-01-04  5.793883
          2000-01-05  5.420548
          2000-01-06  2.575712
          2000-01-07  4.503078
          2000-01-08  3.401633
     h1_1 2000-01-01  4.314028
          2000-01-02  3.674665
          2000-01-03  3.996853
          2000-01-04  3.886664
          2000-01-05  5.047069
          2000-01-06  3.347832
          2000-01-07  3.866058
          2000-01-08  2.698894
     h1_2 2000-01-01  5.822744
          2000-01-02  2.098624
          2000-01-03  3.598748
          2000-01-04  3.365806
          2000-01-05  5.085769
          2000-01-06  5.022349
          2000-01-07  3.707937
          2000-01-08  3.931152
h0_1 h1_0 2000-01-01  4.783280
          2000-01-02  4.755370
          2000-01-03  3.165663
          2000-01-04  3.250687
          2000-01-05  2.504437
          2000-01-06  2.132972
          2000-01-07  1.846720
          2000-01-08  5.503765
     h1_1 2000-01-01  1.939092
          2000-01-02  3.340250
          2000-01-03  2.657523
          2000-01-04  3.939892
          2000-01-05  3.042185
          2000-01-06  2.372358
          2000-01-07  3.524808
          2000-01-08  3.981322
     h1_2 2000-01-01  2.880529
          2000-01-02  3.193437
          2000-01-03  2.739844
          2000-01-04  1.826707
          2000-01-05  3.730416
          2000-01-06  3.151209
          2000-01-07  1.922791
          2000-01-08  4.015772

## Multivariate

In [7]:
forecaster = NaiveForecaster(strategy="last")
forecaster.fit(y_multivariate, fh=[1, 2, 3])

NaiveForecaster()

In [8]:
forecaster.forecasters_

,0,1,2,3
forecasters,NaiveForecaster(),NaiveForecaster(),NaiveForecaster(),NaiveForecaster()


In [18]:
for train, test in cv.split_series(y_multivariate):
    print(f"TRAIN: \n {train}\n TEST: \n {test}")
    print("=====================================")

TRAIN: 
                    0         1         2         3
2000-01-01  5.317042  3.380954  3.685008  4.094989
2000-01-02  5.420548  2.003519  3.656359  1.702739
2000-01-03  3.449771  3.391395  2.850314  3.308369
2000-01-04  4.314028  3.102471  3.150133  2.187770
2000-01-05  5.047069  2.775638  3.019338  1.000000
2000-01-06  1.000000  3.634415  3.570706  1.111931
2000-01-07  5.822744  1.526431  2.752029  1.666912
2000-01-08  5.085769  4.450155  2.861218  2.232258
 TEST: 
                    0         1         2         3
2000-01-09  2.665204  1.000000  2.358358  2.010445
2000-01-10  4.783280  4.183176  2.318943  1.551793
TRAIN: 
                    0         1         2         3
2000-01-01  5.317042  3.380954  3.685008  4.094989
2000-01-02  5.420548  2.003519  3.656359  1.702739
2000-01-03  3.449771  3.391395  2.850314  3.308369
2000-01-04  4.314028  3.102471  3.150133  2.187770
2000-01-05  5.047069  2.775638  3.019338  1.000000
2000-01-06  1.000000  3.634415  3.570706  1.111931
2000

In [31]:
multivariate_scorer = MeanSquaredError(multioutput="raw_values")
cv_errors = manual_evaluate(
    single, y_multivariate, fh_timedelta, forecaster, multivariate_scorer
)

window/fold 0: [4.11026476 4.78402998 1.24432371 3.12115714]


In [25]:
backtest = evaluate(
    forecaster=forecaster,
    y=y_multivariate,
    cv=single,
    scoring=multivariate_scorer,
    return_data=True,
    error_score="raise",
)
backtest

,test_MeanSquaredError,fit_time,pred_time,len_train_window,cutoff,y_train,y_test,y_pred
0,"[4.110264763336158, 4.7840299836879066, 1.2443...",0.050207,0.154773,8,2000-01-10,0 1 2 ...,0 1 2 ...,0 1 2 ...


In [33]:
backtest["y_train"][0]

,0,1,2,3
2000-01-03,3.449771,3.391395,2.850314,3.308369
2000-01-04,4.314028,3.102471,3.150133,2.187770
2000-01-05,5.047069,2.775638,3.019338,1.000000
2000-01-06,1.000000,3.634415,3.570706,1.111931
2000-01-07,5.822744,1.526431,2.752029,1.666912
2000-01-08,5.085769,4.450155,2.861218,2.232258
2000-01-09,2.665204,1.000000,2.358358,2.010445
2000-01-10,4.783280,4.183176,2.318943,1.551793


## Univariate w and w/o Exogenous Data

This is quite tricky as most splitter only accept one argument y as input. The only way to get around this and split X as well is by using SameLocSplitter.

In [46]:
X = y_multivariate.iloc[:, 1:]

from sktime.forecasting.model_selection import SameLocSplitter

cv_X = SameLocSplitter(sliding, y)

for train, test in cv_X.split_series(X):
    print(f"TRAIN: \n {train}\n TEST: \n {test}")
    print("=====================================")

TRAIN: 
                    1         2         3
2000-01-01  3.380954  3.685008  4.094989
2000-01-02  2.003519  3.656359  1.702739
2000-01-03  3.391395  2.850314  3.308369
2000-01-04  3.102471  3.150133  2.187770
2000-01-05  2.775638  3.019338  1.000000
2000-01-06  3.634415  3.570706  1.111931
2000-01-07  1.526431  2.752029  1.666912
2000-01-08  4.450155  2.861218  2.232258
 TEST: 
                    1         2         3
2000-01-09  1.000000  2.358358  2.010445
2000-01-10  4.183176  2.318943  1.551793
TRAIN: 
                    1         2         3
2000-01-03  3.391395  2.850314  3.308369
2000-01-04  3.102471  3.150133  2.187770
2000-01-05  2.775638  3.019338  1.000000
2000-01-06  3.634415  3.570706  1.111931
2000-01-07  1.526431  2.752029  1.666912
2000-01-08  4.450155  2.861218  2.232258
2000-01-09  1.000000  2.358358  2.010445
2000-01-10  4.183176  2.318943  1.551793
 TEST: 
                    1         2         3
2000-01-11  1.560779  1.000000  3.804871
2000-01-12  2.542722 

In [47]:
for train, test in sliding.split_series(y):
    print(f"TRAIN: \n {train}\n TEST: \n {test}")
    print("=====================================")

TRAIN: 
 2000-01-01    5.317042
2000-01-02    5.420548
2000-01-03    3.449771
2000-01-04    4.314028
2000-01-05    5.047069
2000-01-06    1.000000
2000-01-07    5.822744
2000-01-08    5.085769
Freq: D, Name: 0, dtype: float64
 TEST: 
 2000-01-09    2.665204
2000-01-10    4.783280
Freq: D, Name: 0, dtype: float64
TRAIN: 
 2000-01-03    3.449771
2000-01-04    4.314028
2000-01-05    5.047069
2000-01-06    1.000000
2000-01-07    5.822744
2000-01-08    5.085769
2000-01-09    2.665204
2000-01-10    4.783280
Freq: D, Name: 0, dtype: float64
 TEST: 
 2000-01-11    2.504437
2000-01-12    3.043338
Freq: D, Name: 0, dtype: float64


In [57]:
errors = []
mse = MeanSquaredError()
for (train_y, test_y), (train_x, future_x) in zip(
    sliding.split_series(y), cv_X.split_series(X)
):
    forecaster.fit(train_y, train_x)
    y_pred = forecaster.predict(fh, future_x)
    error = mse(test_y, y_pred)
    errors.append(error)

for i, error in enumerate(errors):
    print(f"window/fold {i}: {error}")

window/fold 0: 0.15905576081692285
window/fold 1: 0.5801545236174498


In [58]:
backtest = evaluate(
    forecaster=forecaster,
    y=y,
    X=X,
    cv=sliding,
    scoring=mse,
    return_data=True,
    error_score="raise",
)

# should evaluate also return X_train, X_future if there exists
backtest

,test_MeanSquaredError,fit_time,pred_time,len_train_window,cutoff,y_train,y_test,y_pred
0,0.159056,0.006003,0.029476,8,2000-01-08,2000-01-01 3.741330 2000-01-02 2.377435 ...,2000-01-09 1.874059 2000-01-10 2.387876 ...,2000-01-09 1.825921 2000-01-10 1.825921 ...
1,0.580155,0.007115,0.040105,8,2000-01-10,2000-01-03 2.956016 2000-01-04 4.218171 ...,2000-01-11 2.121321 2000-01-12 3.431551 ...,2000-01-11 2.387876 2000-01-12 2.387876 ...


In [57]:
backtest["y_train"][0]

2000-01-01    5.317042
2000-01-02    5.420548
2000-01-03    3.449771
2000-01-04    4.314028
2000-01-05    5.047069
2000-01-06    1.000000
2000-01-07    5.822744
2000-01-08    5.085769
2000-01-09    2.665204
2000-01-10    4.783280
Freq: D, Name: 0, dtype: float64

## Panel Forecasting

In [6]:
for train, test in cv.split_series(y_panel):
    print(f"TRAIN: \n {train}\n TEST: \n {test}")
    print("=====================================")

TRAIN: 
                          var_0
instances timepoints          
0         0           0.882026
          1           0.200079
          2           0.489369
          3           1.120447
          4           0.933779
          5          -0.488639
          6           0.475044
          7          -0.075679
1         0           0.380519
          1           0.060838
          2           0.221932
          3           0.166837
          4           0.747040
          5          -0.102579
          6           0.156534
          7          -0.427048
2         0           1.134877
          1          -0.727183
          2           0.022879
          3          -0.093592
          4           0.766390
          5           0.734679
          6           0.077474
          7           0.189081
3         0           0.615145
          1           0.601190
          2          -0.193663
          3          -0.151151
          4          -0.524276
          5          -0.710009

In [9]:
manual_evaluate(cv, y_panel, fh, forecaster, hierachical_scorer)

window/fold 0: 0.5541215162513804
window/fold 1: 0.29817888390989095


[0.5541215162513804, 0.29817888390989095]

In [15]:
backtest = evaluate(
    forecaster=forecaster, y=y_panel, cv=cv, scoring=hierachical_scorer, return_data=True, error_score="raise"
)
backtest

,test_MeanSquaredError,fit_time,pred_time,len_train_window,cutoff,y_train,y_test,y_pred
0,0.554122,0.080423,0.155080,48,7,var_0 instances timep...,var_0 instances timep...,var_0 instances timep...
1,0.298179,0.085698,0.274724,60,9,var_0 instances timep...,var_0 instances timep...,var_0 instances timep...


In [16]:
backtest.loc[1, "y_train"]

var_0
instances timepoints          
0         0           0.882026
          1           0.200079
          2           0.489369
          3           1.120447
          4           0.933779
          5          -0.488639
          6           0.475044
          7          -0.075679
          8          -0.051609
          9           0.205299
1         0           0.380519
          1           0.060838
          2           0.221932
          3           0.166837
          4           0.747040
          5          -0.102579
          6           0.156534
          7          -0.427048
          8          -1.276495
          9           0.326809
2         0           1.134877
          1          -0.727183
          2           0.022879
          3          -0.093592
          4           0.766390
          5           0.734679
          6           0.077474
          7           0.189081
          8          -0.443893
          9          -0.990398
3         0           0.615145
          1           0.601190
          2          -0.193663
          3          -0.151151
          4          -0.524276
          5          -0.710009
          6          -0.853135
          7           0.975388
          8          -0.254826
          9          -0.219037
4         0          -0.806949
          1          -0.106370
          2          -0.447733
          3           0.193451
          4          -0.255403
          5          -0.590316
          6          -0.014091
          7           0.214166
          8           0.033259
          9           0.151236
5         0          -0.336230
          1          -0.179777
          2          -0.406573
          3          -0.863141
          4           0.088713
          5          -0.200890
          6          -0.815099
          7           0.231391
          8          -0.453649
          9           0.025973

In [17]:
single_panel = SingleWindowSplitter(fh=fh, window_length=window_length)
backtest = evaluate(
    forecaster=forecaster, y=y_panel, cv=single_panel, scoring=hierachical_scorer, return_data=True, error_score="raise"
)
backtest

,test_MeanSquaredError,fit_time,pred_time,len_train_window,cutoff,y_train,y_test,y_pred
0,0.298179,0.116006,0.286507,48,9,var_0 instances timep...,var_0 instances timep...,var_0 instances timep...


In [18]:
backtest.loc[0, "y_train"]

var_0
instances timepoints          
0         2           0.489369
          3           1.120447
          4           0.933779
          5          -0.488639
          6           0.475044
          7          -0.075679
          8          -0.051609
          9           0.205299
1         2           0.221932
          3           0.166837
          4           0.747040
          5          -0.102579
          6           0.156534
          7          -0.427048
          8          -1.276495
          9           0.326809
2         2           0.022879
          3          -0.093592
          4           0.766390
          5           0.734679
          6           0.077474
          7           0.189081
          8          -0.443893
          9          -0.990398
3         2          -0.193663
          3          -0.151151
          4          -0.524276
          5          -0.710009
          6          -0.853135
          7           0.975388
          8          -0.254826
          9          -0.219037
4         2          -0.447733
          3           0.193451
          4          -0.255403
          5          -0.590316
          6          -0.014091
          7           0.214166
          8           0.033259
          9           0.151236
5         2          -0.406573
          3          -0.863141
          4           0.088713
          5          -0.200890
          6          -0.815099
          7           0.231391
          8          -0.453649
          9           0.025973

## M4 Workflows


In [3]:
from sktime.datasets import load_forecastingdata

data, metadata = load_forecastingdata(
    "m4_weekly_dataset", return_type="pd_multiindex_hier"
)


def data_loader():
    data, _ = load_forecastingdata(
        "m4_weekly_dataset", return_type="pd_multiindex_hier"
    )
    return data

In [4]:
# edit index format see bug #5186
data = data.reset_index()
data["timestamp"] = pd.to_datetime(data["timestamp"].dt.date)
data.set_index(["series_name", "timestamp"], inplace=True)
data

series_value
series_name timestamp               
T1          1975-01-12       1089.20
            1975-01-19       1078.91
            1975-01-26       1079.88
            1975-02-02       1063.58
            1975-02-09       1060.61
...                              ...
T359        2017-09-10       3386.00
            2017-09-17       3627.00
            2017-09-24       3299.00
            2017-10-01       3743.00
            2017-10-08       3965.00

[371579 rows x 1 columns]

### Metrics Evaluation

For each single series, the following metrics are calculated:
$$sMAPE = \frac{2}{n}\sum_{i=1}^{n}\frac{|y_i-\hat{y}_i|}{(|y_i|+|\hat{y}_i|)}$$

$$MASE = \frac{1}{h}\frac{\sum_{t=n+1}^{n+h}|y_t-\hat{y}_t|}{\frac{1}{n-m}\sum_{t=m+1}^{n}|y_t-y_{t-m}|}$$

where $y_i$ is the value of the time series at point $t$, $\hat{y}_i$ the estimated forecast, $h$ the forecasting horizon, $n$ the number of the data points available in-sample, and $m$ is time interval between successive observations considered by **the organizers** for each data frequency, i.e., 12 for monthly, four for quarterly, 24 for hourly and one for yearly, weekly and daily data. 

Aggregrted metrics for ranking models are calculated as follows:

$$OWA = \frac{1}{2}\left|\frac{\frac{1}{N}\sum_{i}^{N}sMAPE_{i}}{\frac{1}{N}\sum_{i}^{N}sMAPE_{i, naive2}}+\frac{\frac{1}{N}\sum_{i}^{N}MASE_{i}}{\frac{1}{N}\sum_{i}^{N}MASE_{i,naive2}}\right| $$


where 
1. $sMAPE_{naive}$ and $MASE_{naive}$ are the sMAPE and MASE of the naive forecast 2, respectively. 
2. $sMAPE_{i}$ and $MASE_{i}$ are the sMAPE and MASE of the model i, respectively.
3. N is the number of  sample time series 

In [20]:
fh_npoints = metadata.get("forecast_horizon") #2
print(f"forecast horizon timepoints: {fh_npoints}")

fh = list(range(1, fh_npoints + 1))
metrics = [
    MeanAbsoluteScaledError(multilevel="uniform_average"),
    MeanAbsolutePercentageError(multilevel="uniform_average"),
]
train_test_splitter = ExpandingGreedySplitter(test_size=fh_npoints, folds=1)

forecast horizon timepoints: 13


Models to considers as paper mentions:
1. Naïve 1
2. Naïve S 
3. Naïve 2
4. SES
5. Holt
6. Damped	
7. Theta
8. Comb
9. MLP
10. RNN
11. ETS
12. ARIMA

Update: unable to proceed. there are some many unexcepted errors. Am I using sktime wrongly or there are bugs everywhere?

1. temporal_train_test_split returns wrong splits: does this function work on unequal length series? the test set instances != train set instances. I think this is a bug.
2. NaiveForecaster(strategy="last") returns wrong timestamp predictions compared to test set but they are in equal length. hence unable to compute metrics. for this reason, simple evaluation steps and evaluate not working.

In [48]:
from sktime.forecasting.model_selection import temporal_train_test_split

train, test = temporal_train_test_split(y=data, test_size=13)
train.tail(15)

c:\Users\hazru\miniconda3\envs\sktime-dev-new\lib\site-packages\sktime\datatypes\_utilities.py:41: PerformanceWarning: indexing past lexsort depth may impact performance.
  return X.loc[tuple(list(X.index[0])[:-1])].index


series_value
series_name timestamp                        
T359        2017-07-02 12:00:00        3242.0
            2017-07-09 12:00:00        4410.0
            2017-07-16 12:00:00        4458.0
            2017-07-23 12:00:00        5098.0
            2017-07-30 12:00:00        4518.0
            2017-08-06 12:00:00        4973.0
            2017-08-13 12:00:00        3973.0
            2017-08-20 12:00:00        2613.0
            2017-08-27 12:00:00        3476.0
            2017-09-03 12:00:00        4213.0
            2017-09-10 12:00:00        3386.0
            2017-09-17 12:00:00        3627.0
            2017-09-24 12:00:00        3299.0
            2017-10-01 12:00:00        3743.0
            2017-10-08 12:00:00        3965.0

In [49]:
test.tail(15)

series_value
series_name timestamp                        
T250        2016-12-18 12:00:00        3232.7
            2016-12-25 12:00:00        3339.3
            2017-01-01 12:00:00        3518.5
T252        2016-10-16 12:00:00        8835.3
            2016-10-23 12:00:00        8859.0
            2016-10-30 12:00:00        8662.7
            2016-11-06 12:00:00        8563.5
            2016-11-13 12:00:00        8897.2
            2016-11-20 12:00:00        8930.5
            2016-11-27 12:00:00        8826.7
            2016-12-04 12:00:00        8675.2
            2016-12-11 12:00:00        8886.2
            2016-12-18 12:00:00        9003.2
            2016-12-25 12:00:00        8917.6
            2017-01-01 12:00:00        8732.1

In [14]:
# Validate for the simplest evaluation case
naive_1 = NaiveForecaster(strategy="last", sp=1)
naive_s = NaiveForecaster(strategy="seasonal_last", sp=1)

cv_errors = manual_evaluate(train_test_splitter, data, fh, naive_1, metrics[1])

window/fold 0: 0.06914815803950736


In [16]:
# NOT WORKING BECAUSE OF ABOVE
# evaluation using evaluate

backtest = evaluate(
    forecaster=naive_1,
    y=data,
    cv=train_test_splitter,
    scoring=metrics, return_data=True, error_score="raise"
)
backtest

In [19]:
backtest.loc[0, "y_pred"]

series_value
series_name timestamp               
T1          2016-12-25     35790.550
            2017-01-01     35790.550
            2017-01-08     35790.550
T2          2016-12-25      3635.781
            2017-01-01      3635.781
...                              ...
T358        2017-10-01      5445.000
            2017-10-08      5445.000
T359        2017-09-24      3627.000
            2017-10-01      3627.000
            2017-10-08      3627.000

[1077 rows x 1 columns]

In [52]:
# ideal case if that allows to run the kotsu benchmarking workflow
# NOT WORKING BECAUSE OF ABOVE
# from sktime.benchmarking.forecasting import ForecastingBenchmark

# benchmark = ForecastingBenchmark()
# benchmark.add_estimator(naive_1, "Naive_1-v1")
# benchmark.add_estimator(naive_s, "Naive_s-v1")

# benchmark.add_task(dataset_loader=data_loader,
#                       cv_splitter=train_test_splitter,
#                       scorers = metrics,
#                       task_id="simpleM4-v1")

# results_df = benchmark.run("forecasting_results.csv")

In [104]:
# Checking splitter outputs
# correct fold to 1
# test has 13 data points for each instance
# ExpandingGreedySplitter does not impliment get_n_splits
for train, test in train_test_splitter.split_series(data):
    # print(f"TRAIN: \n {train}\n TEST: \n {test}")
    print(train.tail(5))
    print(test.tail(15))
    break

                       series_value
series_name timestamp              
T358        85               3660.0
            86               5159.0
            87               6787.0
            88               5596.0
            89               5445.0
...                             ...
T359        85               2613.0
            86               3476.0
            87               4213.0
            88               3386.0
            89               3627.0

[95 rows x 1 columns]


## Datatypes

In [70]:
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.utils._testing.hierarchical import _make_hierarchical

# 6 unique (lowest level) time series, each of length 6
df = _make_hierarchical(
    hierarchy_levels=(2, 3), max_timepoints=6, min_timepoints=6, random_state=0
)

# split each unique time series into 4+2
df_tr_size, df_ts_size = temporal_train_test_split(df, test_size=2)
df

c0
h0   h1   time                
h0_0 h1_0 2000-01-01  5.317042
          2000-01-02  3.953147
          2000-01-03  4.531728
          2000-01-04  5.793883
          2000-01-05  5.420548
          2000-01-06  2.575712
     h1_1 2000-01-01  4.503078
          2000-01-02  3.401633
          2000-01-03  3.449771
          2000-01-04  3.963588
          2000-01-05  3.697033
          2000-01-06  5.007263
     h1_2 2000-01-01  4.314028
          2000-01-02  3.674665
          2000-01-03  3.996853
          2000-01-04  3.886664
          2000-01-05  5.047069
          2000-01-06  3.347832
h0_1 h1_0 2000-01-01  3.866058
          2000-01-02  2.698894
          2000-01-03  1.000000
          2000-01-04  4.206608
          2000-01-05  4.417426
          2000-01-06  2.810825
     h1_1 2000-01-01  5.822744
          2000-01-02  2.098624
          2000-01-03  3.598748
          2000-01-04  3.365806
          2000-01-05  5.085769
          2000-01-06  5.022349
     h1_2 2000-01-01  3.707937
          2000-01-02  3.931152
          2000-01-03  2.665204
          2000-01-04  1.572193
          2000-01-05  3.205078
          2000-01-06  3.709339

In [2]:
from sktime.datatypes import (
    MTYPE_LIST_HIERARCHICAL,
    MTYPE_REGISTER,
    SCITYPE_REGISTER,
    check_is_mtype,
    check_is_scitype,
)

MTYPE_LIST_HIERARCHICAL

array(['pd_multiindex_hier', 'dask_hierarchical'], dtype=object)

In [3]:
SCITYPE_REGISTER

[('Series', 'uni- or multivariate time series'),
 ('Panel', 'panel of uni- or multivariate time series'),
 ('Hierarchical', 'hierarchical panel of time series with 3 or more levels'),
 ('Alignment', 'series or sequence alignment'),
 ('Table', 'data table with primitive column types'),
 ('Proba',
  'probability distribution or distribution statistics, return types')]

In [20]:
MTYPE_REGISTER

[('pd.Series', 'Series', 'pd.Series representation of a univariate series'),
 ('pd.DataFrame',
  'Series',
  'pd.DataFrame representation of a uni- or multivariate series'),
 ('np.ndarray',
  'Series',
  '2D numpy.ndarray with rows=samples, cols=variables, index=integers'),
 ('xr.DataArray',
  'Series',
  'xr.DataArray representation of a uni- or multivariate series'),
 ('dask_series',
  'Series',
  'xdas representation of a uni- or multivariate series'),
 ('nested_univ',
  'Panel',
  'pd.DataFrame with one column per variable, pd.Series in cells'),
 ('numpy3D',
  'Panel',
  '3D np.array of format (n_instances, n_columns, n_timepoints)'),
 ('numpyflat',
  'Panel',
  'WARNING: only for internal use, not a fully supported Panel mtype. 2D np.array of format (n_instances, n_columns*n_timepoints)'),
 ('pd-multiindex',
  'Panel',
  'pd.DataFrame with multi-index (instances, timepoints)'),
 ('pd-wide',
  'Panel',
  'pd.DataFrame in wide format, cols = (instance*timepoints)'),
 ('pd-long',
  '

In [22]:
print(check_is_scitype(y_hierarchical, scitype="Hierarchical"))
print(check_is_mtype(y_hierarchical, mtype="pd_multiindex_hier"))

True

## Tags and Estimator lookup

In [32]:
from sktime.registry import all_estimators, all_tags

all_tags("forecaster", as_dataframe=True)

,name,scitype,type,description
0,X-y-must-have-same-index,"[forecaster, regressor]",bool,do X/y in fit/update and X/fh in predict have ...
1,X_inner_mtype,"[clusterer, forecaster, transformer, transform...","(list, [pd.Series, pd.DataFrame, np.array, nes...",which machine type(s) is the internal _fit/_pr...
2,capability:insample,forecaster,bool,can the forecaster make in-sample predictions?
3,capability:pred_int,forecaster,bool,does the forecaster implement predict_interval...
4,capability:pred_int:insample,forecaster,bool,can the forecaster make in-sample predictions ...
5,capability:pred_var,forecaster,bool,does the forecaster implement predict_variance?
6,enforce_index_type,"[forecaster, regressor]",type,"passed to input checks, input conversion index..."
7,ignores-exogeneous-X,forecaster,bool,does forecaster ignore exogeneous data (X)?
8,remember_data,"[forecaster, transformer]",bool,whether estimator remembers all data seen as s...
9,requires-fh-in-fit,forecaster,bool,does forecaster require fh passed already in f...


In [37]:
all_estimators(
    "forecaster", as_dataframe=True, filter_tags={"scitype:y": "multivariate"}
)

,name,object
0,DynamicFactor,<class 'sktime.forecasting.dynamic_factor.Dyna...
1,VAR,<class 'sktime.forecasting.var.VAR'>
2,VARMAX,<class 'sktime.forecasting.varmax.VARMAX'>
3,VECM,<class 'sktime.forecasting.vecm.VECM'>


## Debugging

# debugging
forecaster = NaiveForecaster(strategy="last", sp=1)  # TrendForecaster()
errors = []
for train, test in train_test_splitter.split_series(data):
    forecaster.fit(train)
    y_pred = forecaster.predict(fh)
    print(f"number of time point {len(y_pred)}")
    print(f"number of time point {len(test)}")
    print("===TAIL===")
    print(test.tail(30))
    print("===")
    print(y_pred.tail(30))
    # print("===HEAD===")
    # print(test.head(15))
    # print("===")
    # print(y_pred.head(15))
#     error = scores(test, y_pred)
#     errors.append(error)

# for i, error in enumerate(errors):
#     print(f"window/fold {i}: {error}")

In [ ]:
import pandas as pd

from sktime.forecasting.model_selection import ExpandingGreedySplitter
from sktime.forecasting.naive import NaiveForecaster

n_instances = 3
n_points = 5

temp = pd.DataFrame(
    {
        "y": [i for i in range(n_points * n_instances)],
        # bugs still occur even if the timestamp is int-like
        "id": ["T1"] * n_points + ["T2"] * n_points + ["T11"] * n_points,
        "timestamp": pd.date_range(
            start="2022-07-01", periods=n_points * n_instances, freq="D"
        ),
    }
).set_index(["id", "timestamp"])

fh = [1, 2]
train_test_splitter = ExpandingGreedySplitter(test_size=len(fh), folds=1)
forecaster = NaiveForecaster(strategy="last")
for train, test in train_test_splitter.split_series(temp):
    forecaster.fit(train)
    y_pred = forecaster.predict(fh)
    print("=====TEST=====")
    print(test.head(15))
    print("=====PREDICT=====")
    print(y_pred.head(15))
    print("=====TRAIN=====")
    print(train.head(15))